# Analysis

This notebook provides several analyses to gain deeper intuition into system performance.

In [ ]:
%matplotlib inline

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pickle
import IPython.display as ipd
import import_ipynb
import system_utils

## Visualizing errors over time

The first analysis is to simply visualize the magnitude of alignment errors over time for a specific concerto movement.

In [ ]:
def visualize_errors_over_time(eval_dir, scenarios_summary, fullmix_id):
    '''
    Visualize the magnitude of alignment error over time for a given full mix recording.  
    Measures that are not evaluated will be displayed as having zero error.
    
    Inputs
    eval_dir: the eval directory containing alignment error information
    scenarios_summary: filepath specifying the scenarios.summary file
    fullmix_id: id specifying the full mix recording of interest, e.g. 'rach2_mov1_PO2'
    '''

    # parse relevant files
    with open(f'{eval_dir}/errs.pkl', 'rb') as f:
        errors_info = pickle.load(f) # key: scenario_id, value: (errs, measNums)
    scenarios_info = system_utils.get_scenario_info(scenarios_summary) # key: scenario_id, value: dict with scenario info
    
    # aggregate data to display
    data = {} # key: tsm_factor, value: dict whose key is the measure number and whose value is the align error
    for scenario_id in scenarios_info:
        if fullmix_id in scenarios_info[scenario_id]['po']:
            tsm_factor = scenarios_info[scenario_id]['p'].split('/')[-2] # e.g. 'tsm0.80'
            errs, measNums = errors_info[scenario_id] # only contains evaluation measures
            if tsm_factor not in data:
                data[tsm_factor] = {}
            for err, measNum in zip(errs, measNums):
                data[tsm_factor][measNum] = err
    
    # display non-evaluated measures as having 0 error
    for tsm_factor in data:
        maxMeasNum = int(np.max(list(data[tsm_factor].keys())))
        for i in range(maxMeasNum):
            if i not in data[tsm_factor]:
                data[tsm_factor][i] = 0
   
    # show alignment error vs measure number
    fig, axs = plt.subplots(len(data.keys()), sharex=True, sharey=True)
    for i, tsm_factor in enumerate(data):
        measures = sorted(data[tsm_factor].keys())
        errors = [data[tsm_factor][m] for m in measures]
        axs[i].plot(measures, errors)
        axs[i].set_title(f'{fullmix_id} - {tsm_factor}')
        axs[i].grid(linestyle='--')
    plt.xlabel('Measure Number')
    plt.ylabel('Alignment Error (sec)')
    plt.ylim([-2.5, 2.5])
    
    return data

In [ ]:
EVAL_DIR = 'eval/offlineDTW' # eval directory to visualize
SCENARIOS_SUMMARY = 'scenarios/scenarios.summary'

In [ ]:
d = visualize_errors_over_time(EVAL_DIR, SCENARIOS_SUMMARY, fullmix_id = 'mozart21_mov1_PO1')

In [ ]:
d = visualize_errors_over_time(EVAL_DIR, SCENARIOS_SUMMARY, fullmix_id = 'mozart21_mov1_PO2')

## Generating sonifications

The second analysis is to sonify estimated alignments.  We generate an audio file that contains an unmodified recording on the left channel and a time-scale modified version of the other recording on the right channel, where time-scale modification is applied in order to synchronize the two recordings.

In [ ]:
import sonify_tools

## Sonifying specific alignments

The cells below generate sonifications of the P-O, P-PO, and PO-O alignments for a given scenario.

In [ ]:
### edit below ###
scenario_id = 's29'
system_name = 'offlineDTW'
downsample = 20
sr = 22050
hop_samples = 512
##################

In [ ]:
### sonifying P-O alignment
outfile = f'{scenario_id}_p_o_align.wav'
hop_len = None # set to None if warping path is already expressed in seconds
audiofile1 = f'scenarios/{scenario_id}/p.wav'
audiofile2 = f'scenarios/{scenario_id}/o.wav'
align_file = f'experiments/{system_name}/{scenario_id}/hyp.npy' # p-o alignment
y = sonify_tools.sonifyWithTSMSync(audiofile1, audiofile2, align_file, downsample, hop_len, outfile)

In [ ]:
### sonifying P-PO alignment
outfile = f'{scenario_id}_p_po_align.wav'
hop_len = hop_samples / sr
audiofile1 = f'scenarios/{scenario_id}/p.wav'
audiofile2 = f'scenarios/{scenario_id}/po.wav'
align_file = f'experiments/{system_name}/{scenario_id}/p_po_align.npy' # p-po alignment
y = sonify_tools.sonifyWithTSMSync(audiofile1, audiofile2, align_file, downsample, hop_len, outfile)

In [ ]:
### sonifying PO-O alignment
outfile = f'{scenario_id}_po_o_align.wav'
hop_len = hop_samples / sr
audiofile1 = f'scenarios/{scenario_id}/po.wav'
audiofile2 = f'scenarios/{scenario_id}/o.wav'
align_file = f'experiments/offlineDTW/cache/mozart21_mov1_O1_PO1/po_o_align.npy' # po-o alignment
y = sonify_tools.sonifyWithTSMSync(audiofile1, audiofile2, align_file, downsample, hop_len, outfile)

Listen to a recording:

In [ ]:
ipd.Audio(f'{scenario_id}_p_po_align.wav')

## Batch Sonification

The following two cells generate P-O sonifications for all scenarios:

In [ ]:
SCENARIOS_DIR = 'scenarios'
EXP_DIR = 'experiments/offlineDTW'
SONIFY_DIR = f'{EXP_DIR}/sonify'
downsample = 20
hop_len = 512./22050

In [ ]:
sonify_tools.sonifyWithTSMSync_batch(SCENARIOS_DIR, EXP_DIR, downsample, hop_len, SONIFY_DIR)